[![UAEM](https://www.uaem.mx/fcaei/images/uaem.png)](https://www.uaem.mx/fcaei/moca.html)
[![Google OR-Tools](https://img.shields.io/badge/Google%20OR--Tools-8A2BE2.svg)](https://developers.google.com/optimization)
[![Python 3.9+](https://img.shields.io/badge/Python-3.9+-blue.svg)](https://www.python.org/downloads/release/python-390/)
[![Maintenance](https://img.shields.io/badge/Maintained%3F-yes-green.svg)](https://github.com/EmilianoRdzV)

# Proyecto: Optimización de Tareas en un Taller (Job Shop Scheduling)

**Autor:** [Emiliano Rodriguez Villegas](https://github.com/EmilianoRdzV)
**Fecha:** 13 de Junio de 2025
**Versión:** 1.0

---

## 1. Introducción y Motivación

Este notebook aborda la resolución de un problema clásico de optimización combinatoria: el **Problema de Planificación de Tareas en un Taller (Job Shop Scheduling Problem - JSSP)**. El objetivo es encontrar la secuencia óptima de operaciones para un conjunto de trabajos que deben procesarse en distintas máquinas, con el fin de minimizar el tiempo total de producción, conocido como **Makespan**.

---

### Índice del Notebook

1.  [**Fase 1: Configuración y Definición del Problema**](#fase-1)
    * [1.1. Introducción al Problema de Job Shop Scheduling](#1-1)
    * [1.2. Instalación y Carga de Bibliotecas](#1-2)
    * [1.3. Definición de los Datos del Problema (Caso 3x3)](#1-3)
2.  [**Fase 2: Modelado y Optimización con OR-Tools**](#fase-2)
    * [2.1. Creación del Modelo y Variables de Decisión](#2-1)
    * [2.2. Implementación de Restricciones (Precedencia y Máquinas)](#2-2)
    * [2.3. Definición del Objetivo (Minimizar Makespan) y Resolución](#2-3)
3.  [**Fase 3: Análisis e Interpretación de Resultados**](#fase-3)
    * [3.1. Extracción y Presentación de la Solución Óptima](#3-1)
    * [3.2. Creación de una Tabla de Planificación con Pandas](#3-2)
    * [3.3. Visualización del Diagrama de Gantt](#3-3)
4.  [**Fase 4: Conclusiones y Pasos Futuros**](#fase-4)
    * [4.1. Discusión de la Solución Encontrada](#4-1)
    * [4.2. Exploración de Problemas Más Complejos](#4-2)

## <a id="fase-1"></a>Fase 1: Configuración y Definición del Problema

### <a id="1-1"></a>1.1. Introducción al Problema de Job Shop Scheduling

El **Job Shop Scheduling Problem (JSSP)** es un problema de optimización en el que un conjunto de **trabajos** (`Jobs`) debe ser procesado en un conjunto de **máquinas** (`Machines`).

Las reglas son las siguientes:
* Cada trabajo consiste en una secuencia de **operaciones** con un orden de precedencia fijo.
* Cada operación debe ser procesada en una máquina específica durante un tiempo determinado.
* Una máquina solo puede procesar una operación a la vez.

El **objetivo** es encontrar una planificación (un cronograma) que minimice el **makespan**: el tiempo total transcurrido desde el inicio del primer trabajo hasta la finalización del último. Este es un problema NP-duro, lo que justifica el uso de solvers especializados.

### <a id="1-2"></a>1.2. Instalación y Carga de Bibliotecas
Primero, nos aseguramos de tener instalada la biblioteca **Google OR-Tools**. Después, importamos el módulo `cp_model`, que es el solver que usaremos.

### <a id="1-3"></a>1.3. Definición de los Datos del Problema (Caso 3x3)
A continuación, definimos nuestro problema específico. Usaremos una lista de listas donde cada sublista representa un trabajo. Dentro de cada trabajo, hay tuplas que representan cada operación en el formato `(id_maquina, tiempo_proceso)`.

**Consideracion:** Para el modelo, las máquinas se indexarán desde 0 (M1=0, M2=1, M3=2).

![Caso de estudio](../Images/casoEstudio.png)

In [3]:
# Formato: jobs_data = [[(máquina, duración), ...], ...]
jobs_data = [
    [(1, 2), (2, 3), (0, 5)],  # Trabajo 1: [O1,1(M2,2), O1,2(M3,3), O1,3(M1,5)]
    [(0, 3), (1, 6), (2, 4)],  # Trabajo 2: [O2,1(M1,3), O2,2(M2,6), O2,3(M3,4)]
    [(1, 2), (0, 2), (2, 4)]   # Trabajo 3: [O3,1(M2,2), O3,2(M1,2), O3,3(M3,4)]
]

num_jobs = len(jobs_data)
num_machines = 3

# Imprimimos los datos para verificar
print(f"Problema definido para {num_jobs} trabajos y {num_machines} máquinas.")
for i, job in enumerate(jobs_data):
    print(f"  Trabajo {i+1}: {job}")

Problema definido para 3 trabajos y 3 máquinas.
  Trabajo 1: [(1, 2), (2, 3), (0, 5)]
  Trabajo 2: [(0, 3), (1, 6), (2, 4)]
  Trabajo 3: [(1, 2), (0, 2), (2, 4)]
